Beam Constraints
================

In some mode matching scenarios, it is not only critical, that the final beam matches the desired mode, the beam must also fit through various other optics along the way. To account for this during mode matching, the relevant optics can be modelled as additional constraints on the beam radius at various positions. The solver will then only consider solutions that satisfy these constraints.

Apertures
---------

The simplest type of constraint is an aperture constraint. It can be used to model optics that are thin compared to the Rayleigh range of the beam such that the beam radius can be approximated as constant over the thickness of the optic. These constraints are modelled using the :class:`~corset.solver.Aperture` class.

An aperture is specified by its position along the optical axis and its radius. Additionally, power fraction can be specified to set a lower limit on the fraction of power that must pass through the aperture. The default value for the power fraction is $1-e^{-2} \approx 0.865$, with this, the specified radius determines the maximum $1/e$ field radius at that point.

In [ ]:
from corset import Aperture

basic_aperture = Aperture(position=0, radius=500e-6)
aperture_with_power = Aperture(position=0, radius=500e-6, power=0.99)

During the optimization process, all apertures are convert to equivalent $1/e$ radius constraints. If the power fraction is larger than $1-e^{-2}$, the equivalent $1/e$ radius is simply decreased to ensure that the specified power fraction passes through the specified aperture radius. 

In [ ]:
print(basic_aperture.radius_constraints) # [(position, radius)]
print(aperture_with_power.radius_constraints) # reduced 1/e radius

Passages
--------

Elements that have a thickness, or in these cases a length, over which the beam radius may significantly change, can be modelled using the :class:`~corset.solver.Passage` class. A passage is specified by its starting (left) and ending (right) position along the optical axis, as well as its radius and an optional power fraction. Alternatively the passage position can also be specified using its center position and width.

In [ ]:
from corset import Passage

simple_passage = Passage(left=0, right=0.1, radius=500e-6)
centered_passage = Passage.centered(center=0.05, width=0.1, radius=500e-6)

Internally, passages are represented as two apertures at the start and end respectively. For this to work, no optics must be placed within the passage, this is enforced by the solver. This also means that the power fraction is only an approximation, that assumes, that only one of the two apertures causes significant clipping of the beam. This should be the case in most situations, but in the unlikely case, that the beam radius is similar at both ends, the actual power loss might be slightly higher than specified. 

In [ ]:
simple_passage.radius_constraints  # [(left, radius), (right, radius)]

Example
-------

Let's look at a minimal example to demonstrate the usage of apertures and passages. Since the initial and final beam are fixed and will be the same for every potential solution, beam constraints only make sense between shifting ranges where the beam shape can actually differ. We will not use power fractions to make it easier to see that the constraint is actually satisfied.

In [ ]:
from IPython.display import display
from corset import Beam, ShiftingRange, ThinLens, mode_match

initial_beam = Beam.from_gauss(focus=0, waist=200e-6, wavelength=1064e-9)
desired_beam = Beam.from_gauss(focus=1, waist=300e-6, wavelength=initial_beam.wavelength)
ranges = [ShiftingRange(0, 0.25), ShiftingRange(0.5, 0.75)]
lenses = [ThinLens(f) for f in [0.1, 0.15, 0.2]]
constraints = [Passage(left=0.3, right=0.4, radius=400e-6), Aperture(position=0.45, radius=300e-6)]

solutions = mode_match(initial_beam, desired_beam, ranges, lenses, max_elements=2, constraints=constraints)
solutions

The constraints are visualized in the setup plot as rectangles and vertical handles for the passages and apertures respectively. The size shown is always the specified radius, not the equivalent $1/e$ radius, so depending on the specified power fraction the beam may never be able to reach the edge or may even surpass it for very low power fractions.

In [ ]:
display(*solutions)

To verify that the constraints were not satisfied by pure luck, we can rerun the search without constraints, and we will see that we will get additional solutions that did not meet the constraints.

In [ ]:
unconstrained_solutions = mode_match(initial_beam, desired_beam, ranges, lenses, max_elements=2)
unconstrained_solutions # one additional solution with an f=100mm and f=200mm lens